In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger

In [36]:
traindata = pd.read_csv('Train.csv')
testdata = pd.read_csv('Test.csv')
print(traindata)
print(testdata)

        duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0              0           tcp  ftp_data   SF        491          0     0   
1              0           udp     other   SF        146          0     0   
2              0           tcp   private   S0          0          0     0   
3              0           tcp      http   SF        232       8153     0   
4              0           tcp      http   SF        199        420     0   
...          ...           ...       ...  ...        ...        ...   ...   
125968         0           tcp   private   S0          0          0     0   
125969         8           udp   private   SF        105        145     0   
125970         0           tcp      smtp   SF       2231        384     0   
125971         0           tcp    klogin   S0          0          0     0   
125972         0           tcp  ftp_data   SF        151          0     0   

        wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0          

In [37]:
encoder = LabelEncoder()
for name in traindata.columns:
  if name in ['protocol_type','service','flag']:
    traindata[name] = encoder.fit_transform(traindata[name])
    testdata[name] = encoder.fit_transform(testdata[name])

In [38]:
traindata

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,49,5,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125969,8,2,49,9,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125970,0,1,54,9,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125971,0,1,30,5,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


#0=Normal, 1=DoS, 2=Probe, 3=R2L, 4=U2R

In [39]:

labeldf=traindata['class']
labeldf_test=testdata['class']


# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})



# put the new label column back
traindata['class'] = newlabeldf
testdata['class'] = newlabeldf_test

In [40]:
traindata

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,49,5,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,1
125969,8,2,49,9,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0
125970,0,1,54,9,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,0
125971,0,1,30,5,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1


In [41]:
testdata

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,45,1,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.00,0.0,1.00,1.00,1
1,0,1,45,1,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,1
2,2,1,19,9,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,0
3,0,0,13,9,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,2
4,1,1,55,2,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,1,49,9,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,0
22540,0,1,22,9,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,0
22541,0,1,22,9,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,1
22542,0,2,11,9,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,0


#X là lấy 41 cột đầu tiên của file traindata
#Y là lấy cột cuối cùng của file traindata
#C là lấy 41 cột đầu tiên của file testdata
#T là lấy cột cuối cùng của file testdata

In [42]:
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]

In [43]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
bestfeatures = SelectKBest(score_func=chi2, k=18)
fit = bestfeatures.fit(X,Y)

In [44]:
# scaler = Normalizer().fit(X)
# trainX = scaler.transform(X)
trainX = fit.transform(X)
trainX

array([[0.0e+00, 2.0e+01, 9.0e+00, ..., 1.7e-01, 0.0e+00, 0.0e+00],
       [0.0e+00, 4.4e+01, 9.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 4.9e+01, 5.0e+00, ..., 1.0e-01, 1.0e+00, 1.0e+00],
       ...,
       [0.0e+00, 5.4e+01, 9.0e+00, ..., 1.2e-01, 7.2e-01, 0.0e+00],
       [0.0e+00, 3.0e+01, 5.0e+00, ..., 3.0e-02, 1.0e+00, 1.0e+00],
       [0.0e+00, 2.0e+01, 9.0e+00, ..., 3.0e-01, 0.0e+00, 0.0e+00]])

In [45]:
# scaler = Normalizer().fit(T)
# testT = scaler.transform(T)
testT = fit.transform(T)
testT

array([[0.0e+00, 4.5e+01, 1.0e+00, ..., 4.0e-02, 0.0e+00, 0.0e+00],
       [0.0e+00, 4.5e+01, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00],
       [2.0e+00, 1.9e+01, 9.0e+00, ..., 6.1e-01, 0.0e+00, 0.0e+00],
       ...,
       [0.0e+00, 2.2e+01, 9.0e+00, ..., 1.0e+00, 0.0e+00, 0.0e+00],
       [0.0e+00, 1.1e+01, 9.0e+00, ..., 9.9e-01, 0.0e+00, 0.0e+00],
       [0.0e+00, 5.2e+01, 1.0e+00, ..., 8.0e-02, 0.0e+00, 0.0e+00]])

In [46]:
Y

0         0
1         0
2         1
3         0
4         0
         ..
125968    1
125969    0
125970    0
125971    1
125972    0
Name: class, Length: 125973, dtype: int64

In [47]:
y_train1 = np.array(Y)
y_train1

array([0, 0, 1, ..., 0, 1, 0])

In [48]:
y_test1 = np.array(C)
y_test1

array([1, 1, 0, ..., 1, 0, 2])

In [49]:
y_train= to_categorical(y_train1)
y_test = to_categorical(y_test1)

In [50]:
y_train

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [51]:
y_test

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

# reshape input to be [samples, time steps, features]

In [52]:
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_train

array([[[0.0e+00, 2.0e+01, 9.0e+00, ..., 1.7e-01, 0.0e+00, 0.0e+00]],

       [[0.0e+00, 4.4e+01, 9.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00]],

       [[0.0e+00, 4.9e+01, 5.0e+00, ..., 1.0e-01, 1.0e+00, 1.0e+00]],

       ...,

       [[0.0e+00, 5.4e+01, 9.0e+00, ..., 1.2e-01, 7.2e-01, 0.0e+00]],

       [[0.0e+00, 3.0e+01, 5.0e+00, ..., 3.0e-02, 1.0e+00, 1.0e+00]],

       [[0.0e+00, 2.0e+01, 9.0e+00, ..., 3.0e-01, 0.0e+00, 0.0e+00]]])

In [53]:
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))
X_test

array([[[0.0e+00, 4.5e+01, 1.0e+00, ..., 4.0e-02, 0.0e+00, 0.0e+00]],

       [[0.0e+00, 4.5e+01, 1.0e+00, ..., 0.0e+00, 0.0e+00, 0.0e+00]],

       [[2.0e+00, 1.9e+01, 9.0e+00, ..., 6.1e-01, 0.0e+00, 0.0e+00]],

       ...,

       [[0.0e+00, 2.2e+01, 9.0e+00, ..., 1.0e+00, 0.0e+00, 0.0e+00]],

       [[0.0e+00, 1.1e+01, 9.0e+00, ..., 9.9e-01, 0.0e+00, 0.0e+00]],

       [[0.0e+00, 5.2e+01, 1.0e+00, ..., 8.0e-02, 0.0e+00, 0.0e+00]]])

In [54]:
batch_size = 32

#Define the network

In [55]:
model = Sequential()
model.add(LSTM(18,input_dim=18))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint("checkpoint-{epoch:03d}.hdf5", verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, callbacks=[checkpointer,csv_logger], validation_data=(X_test, y_test))


Epoch 1/10
3922/3937 [============================>.] - ETA: 0s - loss: 0.2843 - accuracy: 0.9207
Epoch 1: val_accuracy improved from -inf to 0.69575, saving model to checkpoint-001.hdf5
3937/3937 [==============================] - 15s 3ms/step - loss: 0.2841 - accuracy: 0.9207 - val_loss: 1.3589 - val_accuracy: 0.6958
Epoch 2/10
3932/3937 [============================>.] - ETA: 0s - loss: 0.1599 - accuracy: 0.9512
Epoch 2: val_accuracy improved from 0.69575 to 0.71181, saving model to checkpoint-002.hdf5
3937/3937 [==============================] - 13s 3ms/step - loss: 0.1598 - accuracy: 0.9512 - val_loss: 1.3792 - val_accuracy: 0.7118
Epoch 3/10
3927/3937 [============================>.] - ETA: 0s - loss: 0.1573 - accuracy: 0.9529
Epoch 3: val_accuracy did not improve from 0.71181
3937/3937 [==============================] - 12s 3ms/step - loss: 0.1574 - accuracy: 0.9529 - val_loss: 1.4366 - val_accuracy: 0.7026
Epoch 4/10
2987/3937 [=====================>........] - ETA: 2s - loss: 

In [ ]:
from keras import models
model = models.load_model('checkpoint-071.hdf5')
loss, accuracy = model.evaluate(X_test, y_test)
print('loss: ' + str(loss), 'accuracy:' + str(accuracy))

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
cm_df = pd.DataFrame(cm,
                     index = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R'], 
                     columns = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R'])
plt.figure(figsize=(15,10))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()

In [ ]:
normal = {}
DoS = {}
Probe = {}
R2L = {}
U2R = {}

list_label = [normal, DoS, Probe, R2L, U2R]
sum_matrix = sum(sum(cm))
for c in range(len(list_label)):
    tp = cm[c,c]
    fp = sum(cm[:,c]) - cm[c,c]
    fn = sum(cm[c,:]) - cm[c,c]
    # tn = sum(np.delete(sum(cm)-cm[c,:],c))
    tn = sum_matrix - tp - fp - fn
    list_label[c]['TPR'] = tp/(tp+fn)
    # recall = tp/(tp+fn)
    list_label[c]['PPV'] = tp/(tp+fp)
    list_label[c]['NPV'] = tn/(tn+fn)
    list_label[c]['FNR'] = 1 - list_label[c]['TPR']
    list_label[c]['FPR'] = fp/(fp+tn)
    list_label[c]['FDR'] = 1 - list_label[c]['PPV']
    list_label[c]['FOR'] = 1 - list_label[c]['NPV']
    list_label[c]['F1_score'] = (2*list_label[c]['PPV']*list_label[c]['TPR'])/(list_label[c]['PPV'] + list_label[c]['TPR'])
list_tt = ['normal', 'DoS', 'Probe', 'R2L', 'U2R']
for c in range(len(list_label)):

  normal = [list_label[c]]
  df = pd.DataFrame(normal)
  print(df)
  df.plot(kind='bar')
  plt.rcParams['figure.figsize'] = [10, 5]
  plt.rcParams['figure.dpi'] = 100
  plt.title(label=list_tt[c], color="green")
  plt.xlabel('') 
  plt.xticks([])
  plt.yticks([])
  plt.figure(1)
  plt.plot()